In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import tensorflow as tf

tf.__version__

'2.2.0'

In [ ]:
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Flatten, Input, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

## 資料

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

170500096/170498071 [==============================] - 3s 0us/step
(50000, 32, 32, 3)
(10000, 32, 32, 3)
(50000, 1)
(10000, 1)


In [ ]:
x_train[0]

array([[[ 59,  62,  63],
        [ 43,  46,  45],
        [ 50,  48,  43],
        ...,
        [158, 132, 108],
        [152, 125, 102],
        [148, 124, 103]],

       [[ 16,  20,  20],
        [  0,   0,   0],
        [ 18,   8,   0],
        ...,
        [123,  88,  55],
        [119,  83,  50],
        [122,  87,  57]],

       [[ 25,  24,  21],
        [ 16,   7,   0],
        [ 49,  27,   8],
        ...,
        [118,  84,  50],
        [120,  84,  50],
        [109,  73,  42]],

       ...,

       [[208, 170,  96],
        [201, 153,  34],
        [198, 161,  26],
        ...,
        [160, 133,  70],
        [ 56,  31,   7],
        [ 53,  34,  20]],

       [[180, 139,  96],
        [173, 123,  42],
        [186, 144,  30],
        ...,
        [184, 148,  94],
        [ 97,  62,  34],
        [ 83,  53,  34]],

       [[177, 144, 116],
        [168, 129,  94],
        [179, 142,  87],
        ...,
        [216, 184, 140],
        [151, 118,  84],
        [123,  92,  72]]

## 前處理

In [ ]:
# 歸一化

x_train = (x_train - 255.) / 255.
x_test = (x_test - 255.) / 255.

# reshape



In [ ]:
# y onehotencoder
CLASS_NUMS = 10

# one-hot y-data
y_train = tf.keras.utils.to_categorical(y_train, CLASS_NUMS)
y_test = tf.keras.utils.to_categorical(y_test, CLASS_NUMS)

print(y_train.shape)
print(y_test.shape)

(50000, 10)
(10000, 10)


## Model
- AlexNet(5 CNN + 3 Dense)
    - ReLU
    - Dropout
    - BN (optional)
    - Flatten or GAP2D

In [ ]:
# AlexNet(5 CNN + 3 Dense) + Flatten

alexnet = Sequential()

# 5 CNN
alexnet.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(32, 32, 3), activation='relu'))
alexnet.add(BatchNormalization())           
alexnet.add(Conv2D(32, (3, 3), padding='same',
                 activation='relu'))   
alexnet.add(BatchNormalization())           
alexnet.add(Conv2D(64, (3, 3), padding='same',
                 activation='relu'))   
alexnet.add(BatchNormalization())           
alexnet.add(Conv2D(64, (3, 3), padding='same',
                 activation='relu')) 
alexnet.add(BatchNormalization())           
alexnet.add(Conv2D(64, (3, 3), padding='same',
                 activation='relu')) 
alexnet.add(BatchNormalization())           
alexnet.add(MaxPooling2D())

# 3 Dense

alexnet.add(Flatten())
alexnet.add(Dense(128, activation='relu'))
alexnet.add(Dropout(rate=0.2))
alexnet.add(Dense(64, activation='relu'))
alexnet.add(Dropout(rate=0.2))
alexnet.add(Dense(10, activation='softmax'))
alexnet.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_33 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_35 (Conv2D)           (None, 32, 32, 64)        18496     
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
conv2d_36 (Conv2D)           (None, 32, 32, 64)       

In [ ]:
# 超參數

EPOCHS = 10             # 全部樣本丟進去跑10次
BATCH_SIZE = 128        # 一次丟128個樣本

In [ ]:
alexnet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

alexnet.fit(x_train, y_train,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_data=(x_test, y_test))

Epoch 1/10
391/391 [==============================] - 15s 39ms/step - loss: 1.8712 - acc: 0.3423 - val_loss: 1.6213 - val_acc: 0.4212
Epoch 2/10
391/391 [==============================] - 15s 38ms/step - loss: 1.3386 - acc: 0.5213 - val_loss: 1.2605 - val_acc: 0.5606
Epoch 3/10
391/391 [==============================] - 15s 37ms/step - loss: 1.0939 - acc: 0.6179 - val_loss: 1.0144 - val_acc: 0.6551
Epoch 4/10
391/391 [==============================] - 14s 37ms/step - loss: 0.9398 - acc: 0.6747 - val_loss: 1.0168 - val_acc: 0.6473
Epoch 5/10
391/391 [==============================] - 14s 37ms/step - loss: 0.8256 - acc: 0.7154 - val_loss: 0.9394 - val_acc: 0.6741
Epoch 6/10
391/391 [==============================] - 14s 37ms/step - loss: 0.7295 - acc: 0.7460 - val_loss: 0.7993 - val_acc: 0.7294
Epoch 7/10
391/391 [==============================] - 15s 37ms/step - loss: 0.6403 - acc: 0.7788 - val_loss: 0.7934 - val_acc: 0.7327
Epoch 8/10
391/391 [==============================] - 15s 37ms

In [ ]:
def build_alexnet(filters=[32, 32, 64, 64, 64], denses=[128, 64], flatten=True, input_shape=(32, 32, 3), classes=10, bn=True, dropout=True):
    from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Flatten, Dense, Dropout
    from tensorflow.keras.models import Sequential
    
    alexnet = Sequential()
    # 5 CNN
    for e in filters:
        alexnet.add(Conv2D(e, (3, 3), padding='same',
                           input_shape=input_shape, activation='relu'))
        if bn:
            alexnet.add(BatchNormalization())       
    alexnet.add(MaxPooling2D())

    # 3 Dense
    if flatten:
        alexnet.add(Flatten())
    else:
        alexnet.add(GlobalAveragePooling2D())
    alexnet.add(Dense(denses[0], activation='relu'))
    if dropout:
        alexnet.add(Dropout(rate=0.2))  
    alexnet.add(Dense(denses[1], activation='relu'))
    if dropout:
        alexnet.add(Dropout(rate=0.2))
    alexnet.add(Dense(classes, activation='softmax'))
    alexnet.summary()
    return alexnet

In [ ]:
model = build_alexnet()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 64)        3

In [ ]:
# AlexNet + GlobalAveragePooling


alexnet_gap = Sequential()


# 5 CNN
alexnet_gap.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(32, 32, 3), activation='relu'))
alexnet_gap.add(BatchNormalization())
alexnet_gap.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
alexnet_gap.add(BatchNormalization())
alexnet_gap.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
alexnet_gap.add(BatchNormalization())
alexnet_gap.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
alexnet_gap.add(BatchNormalization())
alexnet_gap.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
alexnet_gap.add(BatchNormalization())
alexnet_gap.add(MaxPooling2D())

# 3 Dense
alexnet_gap.add(GlobalAveragePooling2D())
alexnet_gap.add(Dense(128, activation='relu'))
alexnet_gap.add(Dropout(rate=0.2))
alexnet_gap.add(Dense(64, activation='relu'))
alexnet_gap.add(Dropout(rate=0.2))
alexnet_gap.add(Dense(10, activation='softmax'))
alexnet_gap.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_38 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_5 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization_6 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 32, 32, 64)        18496     
_________________________________________________________________
batch_normalization_7 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 32, 32, 64)       

In [ ]:
alexnet_gap.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

alexnet_gap.fit(x_train, y_train,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_data=(x_test, y_test))

Epoch 1/10
391/391 [==============================] - 14s 37ms/step - loss: 1.5926 - acc: 0.4075 - val_loss: 3.6845 - val_acc: 0.1655
Epoch 2/10
391/391 [==============================] - 14s 37ms/step - loss: 1.1898 - acc: 0.5718 - val_loss: 1.2981 - val_acc: 0.5315
Epoch 3/10
391/391 [==============================] - 14s 37ms/step - loss: 1.0179 - acc: 0.6383 - val_loss: 1.0481 - val_acc: 0.6236
Epoch 4/10
391/391 [==============================] - 14s 37ms/step - loss: 0.9148 - acc: 0.6739 - val_loss: 1.1074 - val_acc: 0.5943
Epoch 5/10
391/391 [==============================] - 14s 36ms/step - loss: 0.8448 - acc: 0.7018 - val_loss: 0.9443 - val_acc: 0.6662
Epoch 6/10
391/391 [==============================] - 14s 36ms/step - loss: 0.7822 - acc: 0.7250 - val_loss: 0.9526 - val_acc: 0.6620
Epoch 7/10
391/391 [==============================] - 14s 36ms/step - loss: 0.7324 - acc: 0.7450 - val_loss: 1.0004 - val_acc: 0.6679
Epoch 8/10
391/391 [==============================] - 14s 36ms

## 根據上面的基本AlexNet 去改進
- filter size變大
    - 相當於特徵擷取變多
    - 結果差不多, 但看起來不太穩定
- Epochs增加(原先10)
    - 增加為20
        - 沒有太大改變
- Dense神經元增加(原先128, 64)
    - 增加為256, 128
        - 沒什麼用

In [ ]:
# filter 變大一些
# 原先32, 32, 64, 64, 64
# 改變成64, 64, 128, 128, 128

alexnet3 = Sequential(name='AlexNet3')


# CNN
alexnet3.add(Conv2D(64, (3, 3), padding='same', activation='relu', input_shape=x_train.shape[1:]))
alexnet3.add(BatchNormalization())
alexnet3.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
alexnet3.add(BatchNormalization())
alexnet3.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
alexnet3.add(BatchNormalization())
alexnet3.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
alexnet3.add(BatchNormalization())
alexnet3.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
alexnet3.add(BatchNormalization())
alexnet3.add(MaxPooling2D())

# Dense
alexnet3.add(Flatten())
alexnet3.add(Dense(128, activation='relu'))
alexnet3.add(Dropout(0.2))
alexnet3.add(Dense(64, activation='relu'))
alexnet3.add(Dropout(0.2))
alexnet3.add(Dense(10, activation='softmax'))

alexnet3.summary()

Model: "AlexNet3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_53 (Conv2D)           (None, 32, 32, 64)        1792      
_________________________________________________________________
batch_normalization_20 (Batc (None, 32, 32, 64)        256       
_________________________________________________________________
conv2d_54 (Conv2D)           (None, 32, 32, 64)        36928     
_________________________________________________________________
batch_normalization_21 (Batc (None, 32, 32, 64)        256       
_________________________________________________________________
conv2d_55 (Conv2D)           (None, 32, 32, 128)       73856     
_________________________________________________________________
batch_normalization_22 (Batc (None, 32, 32, 128)       512       
_________________________________________________________________
conv2d_56 (Conv2D)           (None, 32, 32, 128)       147

In [ ]:
EPOCHS = 10
BATCH_SIZE = 128

alexnet3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
alexnet3.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(x_test, y_test))

Epoch 1/10
391/391 [==============================] - 30s 76ms/step - loss: 2.0688 - acc: 0.2783 - val_loss: 2.2970 - val_acc: 0.2218
Epoch 2/10
391/391 [==============================] - 29s 74ms/step - loss: 1.5438 - acc: 0.4383 - val_loss: 1.5009 - val_acc: 0.5037
Epoch 3/10
391/391 [==============================] - 29s 74ms/step - loss: 1.3101 - acc: 0.5257 - val_loss: 1.1415 - val_acc: 0.6061
Epoch 4/10
391/391 [==============================] - 29s 75ms/step - loss: 1.1716 - acc: 0.5804 - val_loss: 1.4407 - val_acc: 0.5429
Epoch 5/10
391/391 [==============================] - 29s 74ms/step - loss: 1.0541 - acc: 0.6245 - val_loss: 1.0129 - val_acc: 0.6550
Epoch 6/10
391/391 [==============================] - 29s 74ms/step - loss: 0.9474 - acc: 0.6593 - val_loss: 1.0093 - val_acc: 0.6580
Epoch 7/10
391/391 [==============================] - 29s 74ms/step - loss: 0.8432 - acc: 0.7012 - val_loss: 0.8652 - val_acc: 0.7089
Epoch 8/10
391/391 [==============================] - 29s 74ms

In [ ]:
alexnet4 = Sequential(name='AlexNet4')


# CNN
alexnet4.add(Conv2D(64, (3, 3), padding='same', activation='relu', input_shape=x_train.shape[1:]))
alexnet4.add(BatchNormalization())
alexnet4.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
alexnet4.add(BatchNormalization())
alexnet4.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
alexnet4.add(BatchNormalization())
alexnet4.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
alexnet4.add(BatchNormalization())
alexnet4.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
alexnet4.add(BatchNormalization())
alexnet4.add(MaxPooling2D())

# Dense
alexnet4.add(GlobalAveragePooling2D())
alexnet4.add(Dense(128, activation='relu'))
alexnet4.add(Dropout(0.2))
alexnet4.add(Dense(64, activation='relu'))
alexnet4.add(Dropout(0.2))
alexnet4.add(Dense(10, activation='softmax'))

alexnet4.summary()


EPOCHS = 10
BATCH_SIZE = 128

alexnet4.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
alexnet4.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(x_test, y_test))

Model: "AlexNet4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_68 (Conv2D)           (None, 32, 32, 64)        1792      
_________________________________________________________________
batch_normalization_35 (Batc (None, 32, 32, 64)        256       
_________________________________________________________________
conv2d_69 (Conv2D)           (None, 32, 32, 64)        36928     
_________________________________________________________________
batch_normalization_36 (Batc (None, 32, 32, 64)        256       
_________________________________________________________________
conv2d_70 (Conv2D)           (None, 32, 32, 128)       73856     
_________________________________________________________________
batch_normalization_37 (Batc (None, 32, 32, 128)       512       
_________________________________________________________________
conv2d_71 (Conv2D)           (None, 32, 32, 128)       147

In [ ]:
## epochs 改變

EPOCHS = 20
BATCH_SIZE = 128

model_20 = build_alexnet()

model_20.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model_20.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(x_test, y_test))

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_5 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization_6 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
batch_normalization_7 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 32, 32, 64)       

## 神經元增加
- 效果不好


In [ ]:
## 神經元 

EPOCHS = 10
BATCH_SIZE = 128

model_256 = build_alexnet(denses=[256, 128])

model_256.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model_256.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(x_test, y_test))

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_10 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization_11 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 32, 32, 64)        18496     
_________________________________________________________________
batch_normalization_12 (Batc (None, 32, 32, 64)        256       
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 32, 32, 64)       

In [ ]:
# 官方文檔的教學


from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

batch_size = 32
num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)

# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Using TensorFlow backend.


x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
Using real-time data augmentation.
Epoch 1/100
1563/1563 [==============================] - 31s 20ms/step - loss: 1.8731 - accuracy: 0.3123 - val_loss: 1.6144 - val_accuracy: 0.4143
Epoch 2/100
1563/1563 [==============================] - 31s 20ms/step - loss: 1.5823 - accuracy: 0.4204 - val_loss: 1.4332 - val_accuracy: 0.4787
Epoch 3/100
1563/1563 [==============================] - 31s 20ms/step - loss: 1.4492 - accuracy: 0.4759 - val_loss: 1.3385 - val_accuracy: 0.5137
Epoch 4/100
1563/1563 [==============================] - 31s 20ms/step - loss: 1.3705 - accuracy: 0.5054 - val_loss: 1.2200 - val_accuracy: 0.5631
Epoch 5/100
1563/1563 [==============================] - 31s 20ms/step - loss: 1.3019 - accuracy: 0.5339 - val_loss: 1.2006 - val_accuracy: 0.5720
Epoch 6/100
1563/1563 [==============================] - 31s 20ms/step - loss: 1.2486 - accuracy: 0.5551 - val_loss: 1.2184 - val_accuracy: 0.5713
Epoch 7/10

## Data Augmentation

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 超參數
BATCH_SIZE = 128
EPOCHS = 100


# 這是real-time的圖片處理, 所以其實並沒有增加圖片, 但可以BASED ON這樣的方法去類似增加
datagen = ImageDataGenerator(
    featurewise_center=False,       # 使得輸入資料集平均為0 
    samplewise_center=False,        # 使樣本平均為0
    featurewise_std_normalization=False,        # 
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=10,      # 旋轉, 0~180
    width_shift_range=0.1,  # 水平移動比例
    height_shift_range=0.1, # 垂直移動比例
    shear_range=0.,         # random shear
    zoom_range=0.,          # random zoom
    channel_shift_range=0.,  # set range for random channel shifts
    # set mode for filling points outside the input boundaries
    fill_mode='nearest',
    cval=0.,  # value used for fill_mode = "constant"
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False,  # randomly flip images
    # set rescaling factor (applied before any other transformation)
    rescale=None,
    # set function that will be applied on each input
    preprocessing_function=None,
    # image data format, either "channels_first" or "channels_last"
    data_format=None,
    # fraction of images reserved for validation (strictly between 0 and 1)
    validation_split=0.0
)

datagen.fit(x_train)

model_aug_alexnet = build_alexnet()
model_aug_alexnet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model_aug_alexnet.fit_generator(datagen.flow(x_train, y_train,
                                             batch_size=BATCH_SIZE),
                                epochs=EPOCHS,
                                validation_data=(x_test, y_test),
                                workers=4)      # workers 進程數量

# Score trained model.
scores = model_aug_alexnet.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_20 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_20 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization_21 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 32, 32, 64)        18496     
_________________________________________________________________
batch_normalization_22 (Batc (None, 32, 32, 64)        256       
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 32, 32, 64)       

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 超參數
# 特地用來與vgg11比較
BATCH_SIZE = 128
EPOCHS = 20


# 這是real-time的圖片處理, 所以其實並沒有增加圖片, 但可以BASED ON這樣的方法去類似增加
datagen = ImageDataGenerator(
    featurewise_center=False,       # 使得輸入資料集平均為0 
    samplewise_center=False,        # 使樣本平均為0
    featurewise_std_normalization=False,        # 
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=10,      # 旋轉, 0~180
    width_shift_range=0.1,  # 水平移動比例
    height_shift_range=0.1, # 垂直移動比例
    shear_range=0.,         # random shear
    zoom_range=0.,          # random zoom
    channel_shift_range=0.,  # set range for random channel shifts
    # set mode for filling points outside the input boundaries
    fill_mode='nearest',
    cval=0.,  # value used for fill_mode = "constant"
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False,  # randomly flip images
    # set rescaling factor (applied before any other transformation)
    rescale=None,
    # set function that will be applied on each input
    preprocessing_function=None,
    # image data format, either "channels_first" or "channels_last"
    data_format=None,
    # fraction of images reserved for validation (strictly between 0 and 1)
    validation_split=0.0
)

datagen.fit(x_train)

model_aug_alexnet_20 = build_alexnet(denses=[4096, 4096])
model_aug_alexnet_20.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model_aug_alexnet_20.fit_generator(datagen.flow(x_train, y_train,
                                             batch_size=BATCH_SIZE),
                                epochs=EPOCHS,
                                validation_data=(x_test, y_test),
                                workers=4)      # workers 進程數量

# Score trained model.
scores = model_aug_alexnet_20.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 64)        3